<a href="https://colab.research.google.com/github/lagocode/20251125_ForBookConnectChallenge/blob/main/book_recommend_ver1_0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. 초기 파이썬 라이브러리, 모듈 설치**

In [ ]:
pip install streamlit google-generativeai numpy requests beautifulsoup4

In [ ]:
pip install -U google-generativeai

In [ ]:
pip install pandas openpyxl

In [ ]:
!npm install localtunnel

In [2]:
# import streamlit as st
import google.generativeai as genai
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
from datetime import datetime

In [3]:
import pandas as pd
import re
import urllib
import time
import copy

#**2. 도서 대출 이력 정보 데이터, KDC 6판 코드 엑셀 전처리하기**

In [8]:
# ==========================================
# 1. 파일 로드 및 설정
# ==========================================
# 코랩의 파일 경로 (예: '/content/파일이름.xlsx')
file_path = "/content/sample_data/book_sample_150.xlsx"

try:
    # 엑셀 파일 읽기
    df = pd.read_excel(file_path)
    print("✅ 데이터 로드 완료")
    print(f"   - 전체 크기: {df.shape}")
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")

✅ 데이터 로드 완료
   - 전체 크기: (150, 20)


In [9]:
# ==========================================
# 2. 열 이름 확인
# ==========================================
print("\n📋 [현재 열 목록]")
print(df.columns.tolist())
print(df.head())


📋 [현재 열 목록]
['도서 대여도서관', '도서대여일', '반납예정일', '반납일', '반납연기일', '자료상태', '등록번호', '서명', '저자', '발행자', '청구기호', '반납도서관', '도서 대여 값', '편권차', '도서 대여현황의 고유값', '부록여부', '지역내상호대차상태', 'ISBN', '반납연기횟수', '반납연기 도서관부호']
   도서 대여도서관                도서대여일                반납예정일                  반납일  \
0    111035  2025/01/01 00:00:00  2025/01/15 23:59:59  2025/02/02 09:30:52   
1    111035  2025/01/01 00:00:00  2025/01/15 23:59:59  2025/02/02 09:30:54   
2    111035  2025/01/01 00:00:00  2025/01/15 23:59:59  2025/01/17 11:46:53   
3    121002  2025/01/01 00:00:00  2025/01/31 23:59:59  2025/01/31 00:00:00   
4    126024  2025/01/01 00:00:00  2025/01/15 23:59:59  2025/01/14 00:00:00   

  반납연기일  자료상태          등록번호                                         서명  \
0   NaN     1  CMA000104296                              사랑 사랑 사랑하는 이유   
1   NaN     1  AMA000148390                                    사랑이 다예요   
2   NaN     1  AMA000001129                                     억새풀. 5   
3   NaN     1  BJL000324055  이번 생에 정

In [10]:

# ==========================================
# 3. 특정 열 누락 (열 삭제)
# ==========================================
# 삭제하고 싶은 열 이름들을 리스트에 적어주세요.
cols_to_drop = ['도서 대여도서관', '도서대여일', '반납예정일', '반납연기일', '발행자', '반납도서관',
                '도서 대여 값', '편권차', '도서 대여현황의 고유값', '부록여부', '지역내상호대차상태', 'ISBN',
                '반납연기횟수', '반납연기 도서관부호']

# errors='ignore': 해당 열이 없어도 에러 없이 넘어감
ndf = df.copy()
ndf = ndf.drop(columns=cols_to_drop, errors='ignore')

print(f"\n🗑️ [열 삭제 완료]")
print(f"   - 삭제 시도한 열: {cols_to_drop}")
print(f"   - 남은 열: {ndf.columns.tolist()}")


🗑️ [열 삭제 완료]
   - 삭제 시도한 열: ['도서 대여도서관', '도서대여일', '반납예정일', '반납연기일', '발행자', '반납도서관', '도서 대여 값', '편권차', '도서 대여현황의 고유값', '부록여부', '지역내상호대차상태', 'ISBN', '반납연기횟수', '반납연기 도서관부호']
   - 남은 열: ['반납일', '자료상태', '등록번호', '서명', '저자', '청구기호']


In [11]:
# ==========================================
# 4. 특정 열의 텍스트 자르기 (조건부 슬라이싱)
# ==========================================
target_column = '청구기호'

def slice_left_3(text):
    r"""
    . 이나 - 를 찾아 그 인덱스 기준 왼쪽 3글자를 반환하는 함수
    예: '813.6' -> '813', '1234.5' -> '234', 'AB-99' -> 'AB'
    """
    text = str(text)

    # 1. 특수문자 위치 탐색
    idx_dot = text.find('.')
    idx_dash = text.find('-')

    # 2. 유효한 인덱스(찾은 것)만 모으기
    found_indices = [i for i in [idx_dot, idx_dash] if i != -1]

    if found_indices:
        # 3. 가장 먼저 나오는 특수문자 위치 선택
        target_idx = min(found_indices)

        # 4. 해당 위치 기준 왼쪽 3글자 슬라이싱
        # target_idx - 3이 음수면 0(처음)부터 시작
        start_idx = max(0, target_idx - 3)
        return text[start_idx:target_idx]
    else:
        # 5. 특수문자가 없으면 그냥 맨 앞 3글자 반환
        return text[:3]

if target_column in ndf.columns:
    # apply 함수를 사용해 각 셀에 로직 적용
    ndf[target_column] = ndf[target_column].apply(slice_left_3)

    print(f"\n✂️ [글자 수 자르기 완료]")
    print(f"   - 대상 열: '{target_column}'")
    print(f"   - 결과 미리보기:\n{ndf[[target_column]].head()}")
else:
    print(f"\n⚠️ '{target_column}' 열이 데이터에 존재하지 않습니다.")


✂️ [글자 수 자르기 완료]
   - 대상 열: '청구기호'
   - 결과 미리보기:
  청구기호
0  843
1  811
2  813
3  513
4  811


In [12]:
# ==========================================
# 5. 추가 슬라이싱 (숫자 뒤 띄어쓰기 기준 절삭)
# ==========================================
# 적용할 열 이름을 지정하세요 (테스트용으로 '반납일' 사용)
target_column_space = '반납일'

def slice_digit_and_space(text):
    r"""
    숫자(\d) 다음에 공백(\s)이 오면, 공백부터 그 뒤를 모두 삭제
    예: 'Room 1 A' -> 'Room 1', 'Shelf 22 B' -> 'Shelf 22'
    """
    text = str(text)
    # 정규식: 숫자(\d) 뒤에 공백(\s)이 있는 패턴 검색
    match = re.search(r'\d\s', text)
    if match:
        # match.start()는 숫자의 시작 인덱스
        # 숫자까지는 살려야 하므로 start() + 1 까지 슬라이싱
        return text[:match.start() + 1]
    return text

if target_column_space in df.columns:
    ndf[target_column_space] = ndf[target_column_space].apply(slice_digit_and_space)

    print(f"\n✂️ [슬라이싱 2 완료: 숫자 뒤 공백 기준]")
    print(f"   - 대상 열: '{target_column_space}'")
    print(f"   - 결과 미리보기:\n{ndf[[target_column_space]].head()}")
else:
    print(f"\n⚠️ '{target_column_space}' 열이 데이터에 존재하지 않습니다.")


✂️ [슬라이싱 2 완료: 숫자 뒤 공백 기준]
   - 대상 열: '반납일'
   - 결과 미리보기:
          반납일
0  2025/02/02
1  2025/02/02
2  2025/01/17
3  2025/01/31
4  2025/01/14


In [13]:
ndf.head()

,반납일,자료상태,등록번호,서명,저자,청구기호
0,2025/02/02,1,CMA000104296,사랑 사랑 사랑하는 이유,므언 티 반 글 ; 제시카 러브 그림 ; 신형건 옮김,843
1,2025/02/02,1,AMA000148390,사랑이 다예요,김용택 사랑시 선집 ; 김선형 그림,811
2,2025/01/17,1,AMA000001129,억새풀. 5,임선영 저,813
3,2025/01/31,1,BJL000324055,이번 생에 정신과는 처음이라 : 정신과 전문의가 말하는 정신과 사용 설명서,닥터 온실 지음,513
4,2025/01/14,1,BNQ000117704,강(江)이 쓰는 시(詩) : 낙동강 연작 제5 시조집,서태수 지음,811


In [19]:
file_path2 = "/content/sample_data/kdc6list2.xlsx"

try:
    # 엑셀 파일 읽기
    kdc_raw_df = pd.read_excel(file_path2, header=None, names=['기호', '장르명'], dtype={'기호': str})
    print("✅ 데이터 로드 완료")
    print(f"   - 전체 크기: {kdc_raw_df.shape}")
except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path2}")

✅ 데이터 로드 완료
   - 전체 크기: (100, 2)


In [20]:
kdc_raw_df['기호'] = kdc_raw_df['기호'].astype(str)

In [21]:
kdc_raw_df

,기호,장르명
0,000,총류
1,010,도서학 서지학
2,020,문헌정보학
3,030,백과사전
4,040,강연집 수필집 연설문집
...,...,...
95,950,남아메리카
96,960,오세아니아 양극지방
97,970,기타
98,980,지리


In [22]:
import pandas as pd
import re
import os

# ==========================================
# 1. 데이터 준비
# ==========================================

# [A] 사용자의 도서관 대출 데이터 (raw_data)
df_library = pd.DataFrame(ndf)

# [B] 새로 구한 KDC 장르표 로드
df_kdc = pd.DataFrame(kdc_raw_df)

# ==========================================
# 2. KDC 장르 매핑 딕셔너리 생성 (Lookup Table)
# ==========================================
# 기호를 Key로, 장르명을 Value로 변환 (검색 속도 향상)
# 예: {'813': '한국소설', '513': '전기공학/통신'}
kdc_dict = dict(zip(df_kdc['기호'].astype(str), df_kdc['장르명']))

def get_genre_name_advanced(call_number, lookup_dict):
    """
    청구기호(3자리)를 이용해 장르표에서 정확한 장르명을 찾습니다.
    (계층적 조회 로직 적용: 정확히 없으면 상위 분류 검색)
    """
    code = str(call_number).split('.')[0] # 소수점 제거 (혹시 있다면)

    # 1. 정확한 매칭 (Exact Match) - 예: 813 -> 한국소설
    if code in lookup_dict:
        return lookup_dict[code]

    # 2. 십단위 매칭 (Tens Fallback) - 예: 813이 없으면 810(한국문학) 검색
    if len(code) == 3:
        tens_code = code[:2] + "0"
        if tens_code in lookup_dict:
            return lookup_dict[tens_code]

    # 3. 백단위 매칭 (Hundreds Fallback) - 예: 813이 없으면 800(문학) 검색
    if len(code) >= 1:
        hundreds_code = code[0] + "00"
        if hundreds_code in lookup_dict:
            return lookup_dict[hundreds_code]

    return "기타" # 매핑 실패 시

# ==========================================
# 3. 데이터 변환 로직 (Helper Functions)
# ==========================================

def clean_author(text):
    """저자명 정제"""
    text = text.split(';')[0]
    text = re.sub(r'(지음|저|글).*', '', text).strip()
    return text

def generate_initial_keywords(title, genre):
    """제목 단어 + 디테일한 장르명으로 키워드 생성"""
    # 제목에서 특수문자 제거하고 단어 단위로 쪼개기
    words = re.sub(r'[^\w\s]', '', title).split()
    # 장르명이 구체적일수록 벡터 품질이 좋아짐 (예: '한국소설')
    keywords = [genre] + words
    return keywords[:6]

# ==========================================
# 4. 실행: 데이터프레임 -> 라이브러리 DB 리스트
# ==========================================
library_db_list = []

print("🔄 데이터 변환 중...")

for idx, row in df_library.iterrows():
    # A. 고급 장르 매핑 적용
    genre_name = get_genre_name_advanced(row['청구기호'], kdc_dict)

    # B. 저자명 정제
    clean_auth = clean_author(row['저자'])

    # C. 날짜 포맷
    loan_date = row['반납일'].replace('/', '-')

    # D. 키워드 생성
    keywords = generate_initial_keywords(row['서명'], genre_name)

    # E. 결과 조립
    book_item = {
        "id": row['등록번호'],
        "title": row['서명'],
        "author": clean_auth,
        "genre": genre_name,  # 이제 '문학' 대신 '영미소설', '한국시' 등이 들어감
        "class_no": row['청구기호'],
        "last_loan_date": loan_date,
        "keywords": keywords
    }
    library_db_list.append(book_item)

# ==========================================
# 5. 결과 출력
# ==========================================
import json
print("\n### 변환 결과 (복사해서 app.py의 LIBRARY_DB에 붙여넣으세요) ###\n")
print(json.dumps(library_db_list, indent=4, ensure_ascii=False))

# 검증용 출력
print(f"\n[검증] 변환 예시:")
print(f"- 원본 청구기호: {df_library.iloc[0]['청구기호']} -> 변환 장르: {library_db_list[0]['genre']}")

🔄 데이터 변환 중...

### 변환 결과 (복사해서 app.py의 LIBRARY_DB에 붙여넣으세요) ###

[
    {
        "id": "CMA000104296",
        "title": "사랑 사랑 사랑하는 이유",
        "author": "므언 티 반",
        "genre": "영미문학",
        "class_no": "843",
        "last_loan_date": "2025-02-02",
        "keywords": [
            "영미문학",
            "사랑",
            "사랑",
            "사랑하는",
            "이유"
        ]
    },
    {
        "id": "AMA000148390",
        "title": "사랑이 다예요",
        "author": "김용택 사랑시 선집",
        "genre": "한국문학",
        "class_no": "811",
        "last_loan_date": "2025-02-02",
        "keywords": [
            "한국문학",
            "사랑이",
            "다예요"
        ]
    },
    {
        "id": "AMA000001129",
        "title": "억새풀. 5",
        "author": "임선영",
        "genre": "한국문학",
        "class_no": "813",
        "last_loan_date": "2025-01-17",
        "keywords": [
            "한국문학",
            "억새풀",
            "5"
        ]
    },
    {
        "id": "BJL000324055",
        "title": "

#**3. LLM 기반 책 키워드 생성**

1. Google Gemini API(벡터화), Google Books API를 가져와서 '책 키워드(길이: 5)'를 생성합니다.

In [ ]:
# ==========================================
# 0. 설정 및 모델 초기화 (가장 중요)
# ==========================================
try:
    from google.colab import userdata
    API_KEY = userdata.get('GEMINI_API_KEY')
except:
    API_KEY = ""

if not API_KEY:
    # 실행 시 입력받거나 여기에 직접 문자열로 입력하세요
    API_KEY = input("Google Gemini API Key를 입력하세요: ").strip()

genai.configure(api_key=API_KEY)

# [수정] model 객체를 전역 변수로 확실하게 생성
try:
    model = genai.GenerativeModel('gemini-2.0-flash')
except Exception as e:
    print(f"⚠️ 모델 초기화 경고: {e}")
    # 구형 라이브러리 대비 Fallback
    model = genai.GenerativeModel('gemini-pro')

In [24]:
# ==========================================
# 2. Google Books API (알라딘 대체)
# ==========================================

def fetch_google_books_description(book_title):
    """
    Google Books API를 이용해 책의 상세 정보를 가져옵니다. (크롤링 아님, API 사용)
    """
    print(f"\n[Google Books] '{book_title}' 정보 검색 중...")

    # Google Books API 엔드포인트
    api_url = "https://www.googleapis.com/books/v1/volumes"
    params = {
        "q": book_title,
        "maxResults": 1,
        "printType": "books",
        "langRestrict": "ko" # 한국어 결과 우선
    }

    try:
        res = requests.get(api_url, params=params)
        data = res.json()

        if "items" in data and len(data["items"]) > 0:
            book_info = data["items"][0]["volumeInfo"]
            title = book_info.get("title", "제목 없음")
            desc = book_info.get("description", "줄거리 정보가 없습니다.")
            authors = ", ".join(book_info.get("authors", []))

            print(f" -> 찾은 책: {title} ({authors})")
            return desc
        else:
            print(" -> Google Books 결과가 없습니다.")
            return None

    except Exception as e:
        print(f" -> API 오류: {e}")
        return None

# [수정됨] genre 인자 추가하여 프롬프트에 반영
def llm_extract_keywords_from_description(model, book_title, description, genre=None):
    """
    책 줄거리(Description)를 LLM에게 주고 핵심 키워드 추출
    """
    print(f"[LLM Thinking] '{book_title}'의 줄거리를 분석해 키워드 추출 중...")

    content_text = description if description else f"{book_title} (줄거리 정보 없음, 일반적 지식 활용)"

    # [추가] 장르와 겹치지 않게 지시
    avoid_instruction = f"단, 장르명인 '{genre}' 자체는 키워드 목록에 포함하지 마." if genre else ""

    prompt = f"""
    책 제목: {book_title}
    책 줄거리/설명:
    {content_text}

    위 내용을 바탕으로 이 책의 특징을 가장 잘 설명하는 '핵심 키워드' 5개를 추출해줘.
    형용사나 명사 형태로. {avoid_instruction}

    결과는 JSON 리스트로 줘: ["키워드1", "키워드2", "키워드3", "키워드4", "키워드5"]
    """
    try:
        res = model.generate_content(prompt, generation_config={"response_mime_type": "application/json"})
        return json.loads(res.text)
    except Exception as e:
        print(f" -> 키워드 추출 실패: {e}")
        return ["정보없음", "일반", "도서", "분석불가", "오류"]

# ==========================================
# 3. 벡터 및 유사도 함수
# ==========================================

def get_embedding(text):
    try:
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=text,
            task_type="semantic_similarity"
        )
        return np.array(result['embedding'])
    except Exception as e:
        print(f"❌ 임베딩 오류: {e}")
        return np.zeros(768)

def cosine_similarity(vec1, vec2):
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return np.dot(vec1, vec2) / (norm1 * norm2)

# ==========================================
# [수정] 가중치 적용 벡터 로직
# ==========================================

def calculate_kdc_score(code1, code2):
    """
    KDC 청구기호(문자열)를 비교하여 계층적 유사도 점수를 반환
    - code1, code2: 예) "813.6", "800"
    """
    # 비교를 위해 문자열로 변환하고 앞뒤 공백 제거
    c1 = str(code1).strip()
    c2 = str(code2).strip()

    # 길이가 짧은 쪽을 기준으로 비교 (최소 1자리 이상이어야 함)
    min_len = min(len(c1), len(c2))
    if min_len == 0: return 0.0

    # [1단계] 3자리(대분류+중분류+소분류) 일치: 같은 세부 장르 (예: 813 == 813)
    # 보통 KDC는 3자리가 기본이므로 3자리까지 확인
    if len(c1) >= 3 and len(c2) >= 3 and c1[:3] == c2[:3]:
        return 1.5  # 강력한 가중치 (벡터 유사도 1.0보다 훨씬 큼)

    # [2단계] 2자리(대분류+중분류) 일치: 비슷한 장르 (예: 813 vs 811 -> 한국문학 내)
    if len(c1) >= 2 and len(c2) >= 2 and c1[:2] == c2[:2]:
        return 1.0  # 중간 가중치

    # [3단계] 1자리(대분류) 일치: 같은 계열 (예: 813 vs 843 -> 문학)
    if c1[:1] == c2[:1]:
        return 0.5  # 약한 가중치

    # 불일치
    return 0.0

def get_content_vector_only(keywords):
    """
    장르를 제외하고, 오직 '키워드'만으로 벡터를 생성 (API 1회 호출)
    """
    # 키워드를 하나의 문장으로 만듦
    kw_str = ", ".join(keywords) if keywords else ""
    # 프롬프트에 '키워드 특징'임을 명시
    semantic_text = f"책의 핵심 키워드 및 특징: {kw_str}"

    return get_embedding(semantic_text)

def vectorize_library(library_db):
    """전체 도서관 벡터화 (장르 제외, 키워드만)"""
    print("\n[System] 전체 장서 벡터화 시작 (키워드 전용)...")
    vectors = {}
    for i, book in enumerate(library_db):
        print(f"\r -> 진행률: {i+1}/{len(library_db)}", end="")

        # [수정] 키워드만 벡터화하는 함수 호출
        vectors[book['id']] = get_content_vector_only(book['keywords'])

        # 무료 등급 속도 제한 (안전하게 4초 대기)
        time.sleep(4.0)
    print("\n[System] 벡터화 완료! ✅")
    return vectors

# ==========================================
# [수정] 단일 책 벡터 갱신 함수 (get_weighted_vector 제거 -> get_content_vector_only 사용)
# ==========================================
def update_single_book_vector(vectors, book):
    """
    [최적화] 변경된 책 1권의 벡터만 갱신 (API 절약)
    장르 제외, 오직 키워드 내용으로만 벡터 생성
    """
    print(f"\n[System] '{book['title']}'의 정보가 변경되어 벡터를 갱신합니다...")
    # [수정됨] get_weighted_vector 대신 get_content_vector_only 사용
    vectors[book['id']] = get_content_vector_only(book['keywords'])
    print(" -> 갱신 완료 ✅")

# ==========================================
# 4. 추천 로직 함수
# ==========================================

def llm_update_user_keywords(model, current_keywords, new_review):
    print(f"\n[LLM Thinking] 사용자 리뷰 분석 중... ('{new_review}')")

    prompt = f"""
    기존 사용자 관심 키워드: {current_keywords}
    새로운 사용자 리뷰: "{new_review}"

    1. 새 리뷰가 기존 키워드에 없는 새로운 취향이나 중요한 통찰을 포함하고 있는지 0~10점으로 평가하세요.
    2. 점수가 7점 이상이라면, 기존 키워드 중 가장 관련성이 떨어지는 것을 제거하고 새 특징을 반영한 키워드를 추가하세요.

    결과는 반드시 JSON 형식으로 주세요:
    {{ "score": 점수(숫자), "reason": "점수 이유", "updated_keywords": ["키워드1", "키워드2", "키워드3", "키워드4", "키워드5"] }}
    """
    try:
        res = model.generate_content(prompt, generation_config={"response_mime_type": "application/json"})
        return json.loads(res.text)
    except Exception as e:
        print(f"LLM Error: {e}")
        return {"score": 0, "reason": "Error", "updated_keywords": current_keywords}

def run_recommendation_logic(user_keywords, current_book_info, library_vectors):
    print(f"\n[Algorithm] 하이브리드 추천 실행 (벡터 유사도 + KDC 규칙 점수)...")

    # 1. 사용자 키워드 벡터 생성 (장르 제외)
    user_vector = get_content_vector_only(user_keywords)

    # 현재 책의 청구기호 (DB에 'class_no' 같은 필드로 숫자 코드가 있어야 함)
    # 만약 DB 필드명이 'genre_code'라면 그것을 사용하세요.
    # 예시 데이터에는 청구기호가 없으므로, 일단 id나 별도 필드를 쓴다고 가정
    current_kdc = current_book_info.get('class_no', '000')

    candidates = []
    for book in LIBRARY_DB:
        if str(book['id']) == str(current_book_info['id']): continue

        # A. 벡터 유사도 (내용 연관성) - 범위: 0.0 ~ 1.0
        vec_score = cosine_similarity(user_vector, library_vectors.get(book['id']))

        # B. KDC 점수 (장르 연관성) - 범위: 0.0, 1.0, 3.0, 5.0
        book_kdc = book.get('class_no', '000') # DB에 있는 청구기호
        kdc_score = calculate_kdc_score(current_kdc, book_kdc)

        # C. 최종 점수 합산
        # 장르 점수가 매우 크므로, 장르가 같으면 무조건 상위권이고
        # 그 안에서 내용(벡터)이 비슷한 순서로 정렬됨
        final_score = vec_score + kdc_score

        candidates.append({
            "book": book,
            "total_score": final_score,
            "vec_score": vec_score,
            "kdc_score": kdc_score
        })

    # 점수순 정렬
    candidates.sort(key=lambda x: x['total_score'], reverse=True)
    top_3 = candidates[:3]

    final_recommendations = []
    for item in top_3:
        # 추천 사유 생성 logic
        reason = ""
        if item['kdc_score'] >= 5.0: reason = "동일 세부 장르 (Best)"
        elif item['kdc_score'] >= 3.0: reason = "비슷한 장르"
        elif item['kdc_score'] >= 1.0: reason = "같은 계열 도서"
        else: reason = "장르는 다르지만 내용이 비슷함 (Unexpected Discovery)"

        final_recommendations.append({
            "type": "Hybrid Pick",
            "book": item['book'],
            "reason": reason,
            "score": item['total_score'],
            "keywords": item['book']['keywords']
        })

    return final_recommendations

In [25]:
# [수정 전] - 문자열로 변환되어 에러 발생
# LIBRARY_DB = json.dumps(library_db_list, indent=4, ensure_ascii=False)

# [수정 후] - 리스트 객체 그대로 사용 (정상 작동)
LIBRARY_DB = copy.deepcopy(library_db_list)

#**4. 책 키워드 -> 벡터 임베딩 과정**

In [26]:
# 1. [최적화] 전체 DB 벡터화는 맨 처음에 딱 한 번만 실행함.
# API 호출을 아끼기 위해, 한 번 벡터화한 초기 벡터 데이터는 엑셀로 변환하여 저장해두고, 저장된 엑셀을 불러와서 재사용하여 API 호출 비용을 감소함.

# library_vectors = vectorize_library(LIBRARY_DB)


[System] 전체 장서 벡터화 시작 (키워드 전용)...
 -> 진행률: 150/150
[System] 벡터화 완료! ✅


In [27]:
#1. [최적화] 전체 DB 벡터화는 맨 처음에 딱 한 번만 실행함.
#전체 DB 벡터화는 api를 사용하기 때문에, 초기 벡터화는 한 번만 시행하고 시연에서는 변환해서 저장해둔 xlsx를 가져와서 초기 벡터 db로 사용함

# print("💾 엑셀 저장을 위해 데이터를 변환 중입니다...")

# # 2. 데이터 변환
# # (Numpy 배열은 엑셀에 바로 저장이 안 되므로, 문자열로 변환해야 합니다)
# excel_data = []
# for book_id, vector in library_vectors.items():
#     excel_data.append({
#         'book_id': book_id,
#         # 벡터 배열을 리스트로 바꾸고 -> JSON 문자열로 변환하여 저장
#         'vector_string': json.dumps(vector.tolist())
#     })

# # 3. 데이터프레임 생성
# df_output = pd.DataFrame(excel_data)

# # 4. 엑셀 파일로 저장
# file_name = "library_vectors.xlsx"
# df_output.to_excel(file_name, index=False)

# print(f"✅ '{file_name}' 파일로 저장 완료! (다운로드하여 시연에 사용하세요)")

💾 엑셀 저장을 위해 데이터를 변환 중입니다...
✅ 'library_vectors.xlsx' 파일로 저장 완료! (다운로드하여 시연에 사용하세요)


In [31]:
import numpy as np
import os

def load_vectors_from_excel(file_path="library_vectors.xlsx"):
    """
    엑셀 파일에 저장된 문자열 형태의 벡터를 읽어와서,
    시스템이 계산 가능한 {book_id: numpy_array} 형태의 딕셔너리로 복원합니다.
    """
    if not os.path.exists(file_path):
        print(f"❌ 오류: '{file_path}' 파일이 없습니다. 먼저 벡터를 생성하고 저장해주세요.")
        return {}

    print(f"📂 '{file_path}'에서 벡터 데이터를 불러오는 중...")

    try:
        # 1. 엑셀 읽기
        df = pd.read_excel(file_path)

        restored_vectors = {}

        for _, row in df.iterrows():
            book_id = row['book_id']
            vector_str = row['vector_string']

            # 2. 문자열(JSON) -> 리스트 -> Numpy 배열로 변환
            # (저장할 때 json.dumps로 풀었으니, json.loads로 다시 묶고 np.array로 만듭니다)
            vector_list = json.loads(vector_str)
            vector_array = np.array(vector_list)

            # 3. 딕셔너리에 저장
            # 주의: 엑셀은 숫자를 자동으로 정수/실수로 변환하므로,
            # LIBRARY_DB의 ID가 문자열이라면 str(book_id)로 변환이 필요할 수 있습니다.
            # 여기서는 있는 그대로 로드합니다.
            restored_vectors[book_id] = vector_array

        print(f"✅ 로딩 완료! 총 {len(restored_vectors)}개의 도서 벡터가 준비되었습니다.")
        return restored_vectors

    except Exception as e:
        print(f"❌ 벡터 로딩 중 오류 발생: {e}")
        return {}

# ==========================================
# [시연용] 사용 예시
# ==========================================
# 시연 시에는 이 함수를 호출해서 변수에 담으세요.
# library_vectors = load_vectors_from_excel()

# **5. 서비스 실제 시연 파트**

In [ ]:
# ==========================================
# [NEW] 1단계: 초기화 및 데이터 보강 함수 (Run Once)
# ==========================================
def initialize_system(target_book_title):
    print("=========================================")
    print("      [1단계] 시스템 초기화 및 보강")
    print("=========================================")

    # 1. 책 찾기
    found = [b for b in LIBRARY_DB if b['title'] == target_book_title]
    if not found:
        print(f"❌ 책을 찾을 수 없습니다: {target_book_title}")
        return None, None, None
    current_book = found[0]
    print(f"📖 선택된 책: {current_book['title']}")

    # 2. 구글 API 기반 키워드 보강 (이미 보강된 경우 스킵 가능)
    if not current_book.get('enriched', False):
        desc = fetch_google_books_description(current_book['title'])
        # 장르 코드가 있으면 전달, 없으면 genre 필드 사용
        g_code = current_book.get('class_no', current_book.get('genre'))
        new_kws = llm_extract_keywords_from_description(model, current_book['title'], desc, genre=g_code)

        # [중요] 키워드 업데이트: [장르명/코드] + [추출된 키워드]
        # 벡터 생성시엔 내용만 보지만, 리스트엔 장르 정보도 남겨둠
        current_book['keywords'] = [str(g_code)] + new_kws
        current_book['enriched'] = True
        print(f" -> 🌟 보강된 키워드: {current_book['keywords']}")
    else:
        print(" -> (이미 보강된 데이터 사용)")

    # 3. 전체 벡터화 (최초 1회)
    # 전역 변수나 외부에서 주입받는 식으로 처리 권장
    # if 'global_vectors' not in globals():
    #     global_vectors = vectorize_library(LIBRARY_DB)
    # else:
    #     print("\n[System] 기존 벡터 데이터 사용")
    #     global_vectors = globals()['global_vectors']

    # 4. 보강된 책만 벡터 갱신
    update_single_book_vector(global_vectors, current_book)

    return current_book, global_vectors

In [ ]:
# ==========================================
# [NEW] 2단계: 사용자 인터랙션 루프 (Repeatable)
# ==========================================
def run_user_feedback_loop(current_book, vectors):
    print("\n=========================================")
    print("      [2단계] 사용자 피드백 및 추천")
    print("=========================================")

    print(f"📘 읽은 책: {current_book['title']}")
    print(f"🔑 현재 키워드: {current_book['keywords']}")

    review = input("\n💬 후기를 입력하세요 (예: 너무 어려웠어): ")
    if not review: review = "내용은 좋았지만 용어가 어려웠다."

    # 키워드 업데이트
    res = llm_update_user_keywords(model, current_book['keywords'], review)
    final_kws = res['updated_keywords'] if res['score'] >= 7 else current_book['keywords']

    if res['score'] >= 7: print(f" -> ✅ 키워드 변경됨: {final_kws}")
    else: print(f" -> ⏹️ 키워드 유지됨: {final_kws}")

    # 추천 실행
    recs = run_recommendation_logic(final_kws, current_book, vectors)

    print("\n🏆 추천 결과")
    for r in recs:
        print(f"[{r['type']}] {r['book']['title']}")
        print(f" - 이유: {r['reason']}")
        print(f" - 점수: {r['score']:.4f} (Vec {r['vec']:.2f} + KDC {r['kdc']})")
        print(f" - 키워드: {r['book']['keywords']}")
        print("-" * 30)

In [ ]:
# ==========================================
# 5. 메인 실행 (Main) - 셀 분리 가능 구조
# ==========================================
if __name__ == "__main__":
    # 1. 초기화 및 보강 (이 부분만 따로 실행 가능)
    target_title = "주식시장을 이긴 전략들:검증된 방법으로 투자하라!"
    book_obj, lib_vecs = initialize_system(target_title)

    # 2. 루프 실행 (이 부분만 반복 실행 가능)
    if book_obj and lib_vecs:
        run_user_feedback_loop(book_obj, lib_vecs)

In [30]:
# ==========================================
# 5. 실행 (Main) - 최신 알고리즘(Rule+Vector) 반영
# ==========================================
if __name__ == "__main__":
    print("=========================================")
    print("      추천 시스템 로직 검증 모듈")
    print("=========================================")

    # 1. 상황 설정 (책 선택)
    TARGET_BOOK_TITLE = "주식시장을 이긴 전략들:검증된 방법으로 투자하라!"

    # 리스트 컴프리헨션으로 안전하게 찾기
    found_books = [b for b in LIBRARY_DB if b['title'] == TARGET_BOOK_TITLE]
    if not found_books:
        print(f"❌ '{TARGET_BOOK_TITLE}' 책을 DB에서 찾을 수 없습니다.")
        exit()
    current_book = found_books[0]

    print(f"\n[Step 0] 선택된 책: {current_book['title']}")
    print(f" -> 초기 키워드: {current_book['keywords']}")

    # 2. [NEW] Google Books API 기반 키워드 추출 및 업데이트
    description = fetch_google_books_description(current_book['title'])
    # 장르(청구기호)는 키워드 생성 시 제외하도록 요청
    new_extracted_keywords = llm_extract_keywords_from_description(
        model,
        current_book['title'],
        description,
        genre=current_book.get('genre', '') # 혹은 class_no
    )

    # 추출된 키워드로 책 정보 업데이트
    # (장르를 맨 앞에 붙여주되, 벡터 생성시엔 get_content_vector_only가 알아서 처리함)
    current_book['keywords'] = [current_book['genre']] + new_extracted_keywords
    print(f" -> 🌟 업데이트된 키워드: {current_book['keywords']}")

    # 3. [최적화] 전체 재계산 대신, 바뀐 책 1권만 벡터 갱신!
    # (주의: library_vectors 변수가 이미 메모리에 있다고 가정. 없으면 vectorize_library 실행 필요)
    # if 'library_vectors' not in locals():
    #     library_vectors = vectorize_library(LIBRARY_DB)

    update_single_book_vector(library_vectors, current_book)

    # 4. 사용자 후기 입력
    print("\n-----------------------------------------")
    review_input = input("테스트할 후기를 입력하세요 (예: 너무 어렵고 지루했다): ")
    if not review_input: review_input = "초보자가 읽기에는 용어가 너무 어려웠다."

    # 5. 사용자 선호 키워드 업데이트 (LLM)
    update_result = llm_update_user_keywords(model, current_book['keywords'], review_input)

    if update_result['score'] >= 7:
        user_final_keywords = update_result['updated_keywords']
        print(f" -> ✅ 사용자 취향 키워드 변경됨: {user_final_keywords}")
    else:
        user_final_keywords = current_book['keywords']
        print(f" -> ⏹️ 키워드 유지됨: {user_final_keywords}")

    # 6. 최종 추천 로직 실행
    final_recs = run_recommendation_logic(user_final_keywords, current_book, library_vectors)

    # 7. 결과 출력 (키워드 포함)
    print("\n=========================================")
    print("       🎉 최종 추천 결과 (3권)")
    print("=========================================")
    for rec in final_recs:
        print(f"[{rec['type']}] {rec['book']['title']}")
        print(f" - 선정 논리: {rec['reason']}")
        # 점수 디테일 출력 (벡터 점수 + KDC 점수)
        print(f" - 총점: {rec['score']:.4f} (벡터: {rec.get('vec', 0):.4f} + 장르: {rec.get('kdc', 0)})")
        print(f" - 키워드: {rec['keywords']}")
        print("-----------------------------------------")

    # =================================================
    # [Bonus] 키워드 리스트 기반 추천 테스트 (최신 로직 반영)
    # =================================================
    print("\n\n=========================================")
    print("   🧪 [Bonus] 키워드 리스트 기반 추천 테스트")
    print("=========================================")

    input_str = input("원하는 키워드를 쉼표로 구분해 입력 (예: 우주, 과학, 물리학): ")
    if not input_str.strip():
        target_keywords = ["사랑", "이별", "슬픔"]
    else:
        target_keywords = [k.strip() for k in input_str.split(',')]

    target_genre_code = input("선호하는 장르 코드(예: 813, 300) [엔터치면 무시]: ").strip()
    if not target_genre_code: target_genre_code = "000"

    print(f"\n -> 입력 키워드: {target_keywords}")
    print(f" -> 입력 장르코드: {target_genre_code}")

    # 1. 쿼리 벡터 생성 (오직 키워드만 사용)
    query_vector = get_content_vector_only(target_keywords)

    # 2. 점수 계산 (벡터 유사도 + KDC 점수)
    sim_results = []
    for book in LIBRARY_DB:
        # A. 벡터 유사도
        b_vec = library_vectors.get(book['id'])
        vec_score = cosine_similarity(query_vector, b_vec)

        # B. KDC 점수 (입력받은 장르 코드와 비교)
        # DB에 class_no가 있으면 그걸 쓰고, 없으면 genre를 쓰되 숫자가 아니면 0점 처리됨
        book_kdc = book.get('class_no', book.get('genre', '000'))
        kdc_score = calculate_kdc_score(target_genre_code, book_kdc)

        # C. 합산
        total_score = vec_score + kdc_score

        sim_results.append({
            "book": book,
            "total": total_score,
            "vec": vec_score,
            "kdc": kdc_score
        })

    # 3. 정렬 및 출력
    sim_results.sort(key=lambda x: x['total'], reverse=True)

    print(f"\n🏆 키워드 + 장르 복합 추천 Top 3")
    for i, item in enumerate(sim_results[:3]):
        bk = item['book']
        print(f"[{i+1}위] {bk['title']} ({bk['author']})")
        print(f"   - 총점: {item['total']:.4f} (벡터: {item['vec']:.4f} + 장르: {item['kdc']})")
        print(f"   - 장르: {bk.get('genre')} (Code: {bk.get('class_no', 'N/A')})")
        print(f"   - 키워드: {bk['keywords']}")
        print("-----------------------------------------")

      추천 시스템 로직 검증 모듈

[Step 0] 선택된 책: 주식시장을 이긴 전략들:검증된 방법으로 투자하라!
 -> 초기 키워드: ['주식시장', '투자전략', '검증된 방법', '투자', '승리']

[Google Books] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!' 정보 검색 중...
 -> 찾은 책: 주식시장을 이긴 전략들 (박상우)
[LLM Thinking] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!'의 줄거리를 분석해 키워드 추출 중...
 -> 🌟 업데이트된 키워드: ['경제학', '주식시장', '투자', '전략', '검증된 방법', '수익']

[System] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!'의 정보가 변경되어 벡터를 갱신합니다...


NameError: name 'get_weighted_vector' is not defined

In [28]:
# ==========================================
# 5. 실행 (Main) - 최적화 버전
# ==========================================
if __name__ == "__main__":
    print("=========================================")
    print("      추천 시스템 로직 검증 모듈")
    print("=========================================")
    # 2. 상황 설정 (책 선택)
    TARGET_BOOK_TITLE = "주식시장을 이긴 전략들:검증된 방법으로 투자하라!"

    # 리스트 컴프리헨션으로 안전하게 찾기
    found_books = [b for b in LIBRARY_DB if b['title'] == TARGET_BOOK_TITLE]
    if not found_books:
        print(f"❌ '{TARGET_BOOK_TITLE}' 책을 DB에서 찾을 수 없습니다.")
        exit()
    current_book = found_books[0]

    print(f"\n[Step 0] 선택된 책: {current_book['title']}")
    print(f" -> 초기 키워드: {current_book['keywords']}")

    # 3. [NEW] Google Books API 기반 키워드 추출 및 업데이트
    description = fetch_google_books_description(current_book['title'])
    new_extracted_keywords = llm_extract_keywords_from_description(model, current_book['title'], description)

    # 추출된 키워드로 책 정보 업데이트
    current_book['keywords'] = new_extracted_keywords
    print(f" -> 🌟 업데이트된 키워드: {current_book['keywords']}")

    # 4. [최적화] 전체 재계산 대신, 바뀐 책 1권만 벡터 갱신!
    update_single_book_vector(library_vectors, current_book)

    # 5. 사용자 후기 입력
    print("\n-----------------------------------------")
    review_input = input("테스트할 후기를 입력하세요 (예: 너무 어렵고 지루했다): ")
    if not review_input: review_input = "초보자가 읽기에는 용어가 너무 어려웠다."

    # 6. 사용자 선호 키워드 업데이트 (LLM)
    update_result = llm_update_user_keywords(model, current_book['keywords'], review_input)

    if update_result['score'] >= 7:
        user_final_keywords = update_result['updated_keywords']
        print(f" -> ✅ 사용자 취향 키워드 변경됨: {user_final_keywords}")
    else:
        user_final_keywords = current_book['keywords']
        print(f" -> ⏹️ 키워드 유지됨: {user_final_keywords}")

    # 7. 최종 추천 로직 실행
    final_recs = run_recommendation_logic(user_final_keywords, current_book, library_vectors)

    # 8. 결과 출력 (키워드 포함)
    print("\n=========================================")
    print("       🎉 최종 추천 결과 (3권)")
    print("=========================================")
    for rec in final_recs:
        print(f"[{rec['type']}] {rec['title']}")
        print(f" - 선정 논리: {rec['reason']}")
        print(f" - 벡터 유사도: {rec['score']:.4f}")
        print(f" - 키워드: {rec['keywords']}")
        print("-----------------------------------------")

    # =================================================
    # [Bonus] 키워드 리스트 기반 추천 테스트
    # =================================================
    print("\n\n=========================================")
    print("   🧪 [Bonus] 키워드 리스트 기반 추천 테스트")
    print("=========================================")

    input_str = input("원하는 키워드를 쉼표로 구분해 입력 (예: 우주, 과학, 물리학): ")
    if not input_str.strip():
        target_keywords = ["사랑", "이별", "슬픔"]
        target_genre = "영미소설"
    else:
        target_keywords = [k.strip() for k in input_str.split(',')]
        target_genre = input("선호하는 장르(예: 소설, 과학, 역사) [엔터치면 무시]: ").strip()
        if not target_genre: target_genre = "" # 장르 없으면 키워드만

    print(f"\n -> 입력 키워드: {target_keywords}")
    print(f" -> 입력 장르: {target_genre if target_genre else '(없음)'}")

    # 가상 쿼리 벡터 생성 (장르가 없으면 키워드만 벡터화)
    if target_genre:
        query_vector = get_weighted_vector(target_genre, target_keywords)
    else:
        # 장르 없이 키워드만 있는 경우
        query_vector = get_embedding(f"키워드 특징: {', '.join(target_keywords)}")

    # 유사도 계산
    sim_results = []
    for book in LIBRARY_DB:
        b_vec = library_vectors.get(book['id'])
        sc = cosine_similarity(query_vector, b_vec)
        sim_results.append({"book": book, "score": sc})

    sim_results.sort(key=lambda x: x['score'], reverse=True)

    print(f"\n🏆 리스트 기반 추천 Top 3")
    for i, item in enumerate(sim_results[:3]):
        bk = item['book']
        print(f"[{i+1}위] {bk['title']} ({bk['author']})")
        print(f"   - 장르: {bk['genre']} | 점수: {item['score']:.4f}")
        print(f"   - 키워드: {bk['keywords']}")
        print("-----------------------------------------")

      추천 시스템 로직 검증 모듈

[Step 0] 선택된 책: 주식시장을 이긴 전략들:검증된 방법으로 투자하라!
 -> 초기 키워드: ['경제학', '주식시장을', '이긴', '전략들검증된', '방법으로', '투자하라']

[Google Books] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!' 정보 검색 중...
 -> 찾은 책: 주식시장을 이긴 전략들 (박상우)
[LLM Thinking] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!'의 줄거리를 분석해 키워드 추출 중...
 -> 🌟 업데이트된 키워드: ['주식시장', '투자전략', '검증된 방법', '투자', '전략']

[System] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!'의 정보가 변경되어 벡터를 갱신합니다...


NameError: name 'get_weighted_vector' is not defined

In [29]:
# ==========================================
# 5. 실행 (Main)
# ==========================================
if __name__ == "__main__":
    print("=========================================")
    print("      추천 시스템 로직 검증 모듈")
    print("=========================================")

    # 1. 상황 설정 (책 선택)
    TARGET_BOOK_TITLE = "주식시장을 이긴 전략들:검증된 방법으로 투자하라!"

    # [수정된 부분] 리스트 컴프리헨션으로 안전하게 찾기
    found_books = [b for b in LIBRARY_DB if b['title'] == TARGET_BOOK_TITLE]
    if not found_books:
        print(f"❌ '{TARGET_BOOK_TITLE}' 책을 DB에서 찾을 수 없습니다.")
        exit()
    current_book = found_books[0]

    print(f"\n[Step 0] 선택된 책: {current_book['title']}")
    print(f" -> 초기 키워드: {current_book['keywords']}")

    # 2. [NEW] Google Books API 기반 키워드 추출 로직
    description = fetch_google_books_description(current_book['title'])
    new_extracted_keywords = llm_extract_keywords_from_description(model, current_book['title'], description)

    # 추출된 키워드로 현재 책 정보 업데이트
    current_book['keywords'] = new_extracted_keywords
    print(f" -> 🌟 Google Books/LLM 기반 추출 키워드: {current_book['keywords']}")

    # 3. DB 벡터화
    library_vectors = vectorize_library(LIBRARY_DB)

    # 4. 사용자 후기 입력
    print("\n-----------------------------------------")
    review_input = input("테스트할 후기를 입력하세요 (예: 너무 어렵고 지루했다): ")
    if not review_input: review_input = "초보자가 읽기에는 용어가 너무 어려웠다."

    # 5. 사용자 선호 키워드 업데이트
    update_result = llm_update_user_keywords(model, current_book['keywords'], review_input)

    if update_result['score'] >= 7:
        user_final_keywords = update_result['updated_keywords']
        print(f" -> ✅ 사용자 취향 키워드 변경됨: {user_final_keywords}")
    else:
        user_final_keywords = current_book['keywords']
        print(f" -> ⏹️ 키워드 유지됨: {user_final_keywords}")

    # 6. 최종 추천 로직 실행
    final_recs = run_recommendation_logic(user_final_keywords, current_book, library_vectors)

    # 7. 결과 출력
    print("\n=========================================")
    print("       🎉 최종 추천 결과 (3권)")
    print("=========================================")
    for rec in final_recs:
        print(f"[{rec['type']}] {rec['title']}")
        print(f" - 선정 논리: {rec['reason']}")
        print(f" - 벡터 유사도: {rec['score']:.4f}")
        print(f" - 키워드: {rec['keywords']}")
        print("-----------------------------------------")

  # =================================================
    # [추가 기능] 단순 키워드 리스트 기반 추천 테스트
    # =================================================
    print("\n\n=========================================")
    print("   🧪 [Bonus] 키워드 리스트 기반 추천 테스트")
    print("=========================================")

    # 1. 사용자 입력 (리스트 형태)
    input_str = input("원하는 키워드를 쉼표로 구분해 입력 (예: 우주, 과학, 물리학): ")
    if not input_str.strip():
        target_keywords = ["사랑", "이별", "슬픔"] # 기본값
        target_genre = "영미소설" # 기본 장르 (가중치 때문에 필요함)
    else:
        target_keywords = [k.strip() for k in input_str.split(',')]
        target_genre = input("선호하는 장르(예: 소설, 과학, 역사): ").strip()
        if not target_genre: target_genre = "총류"

    print(f"\n -> 입력 키워드: {target_keywords}")
    print(f" -> 입력 장르: {target_genre} (가중치 x5 적용)")

    # 2. 가상 쿼리 벡터 생성
    query_vector = get_weighted_vector(target_genre, target_keywords)

    # 3. 유사도 계산
    sim_results = []
    for book in LIBRARY_DB:
        b_vec = library_vectors.get(book['id'])
        sc = cosine_similarity(query_vector, b_vec)
        sim_results.append({"book": book, "score": sc})

    # 4. 상위 3개 출력
    sim_results.sort(key=lambda x: x['score'], reverse=True)

    print(f"\n🏆 '{target_genre}' 장르 및 키워드 기반 추천 Top 3")
    for i, item in enumerate(sim_results[:3]):
        bk = item['book']
        print(f"[{i+1}위] {bk['title']} ({bk['author']})")
        print(f"   - 장르: {bk['genre']} | 점수: {item['score']:.4f}")
        print(f"   - 키워드: {bk['keywords']}")
        print("-----------------------------------------")

      추천 시스템 로직 검증 모듈

[Step 0] 선택된 책: 주식시장을 이긴 전략들:검증된 방법으로 투자하라!
 -> 초기 키워드: ['주식시장', '투자전략', '검증된 방법', '투자', '전략']

[Google Books] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!' 정보 검색 중...
 -> 찾은 책: 주식시장을 이긴 전략들 (박상우)
[LLM Thinking] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!'의 줄거리를 분석해 키워드 추출 중...
 -> 🌟 Google Books/LLM 기반 추출 키워드: ['주식시장', '투자전략', '검증된 방법', '투자', '승리']

[System] 전체 장서 벡터화 시작 (키워드 전용)...
 -> 진행률: 1/150

KeyboardInterrupt: 

In [ ]:
# =========================================
#       추천 시스템 로직 검증 모듈
# =========================================

# [Step 0] 선택된 책: 주식시장을 이긴 전략들:검증된 방법으로 투자하라!
#  -> 초기 키워드: ['주식시장', '투자전략', '검증된 방법', '투자', '전략']

# [Google Books] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!' 정보 검색 중...
#  -> 찾은 책: 주식시장을 이긴 전략들(개정판) (박상우)
# [LLM Thinking] '주식시장을 이긴 전략들:검증된 방법으로 투자하라!'의 줄거리를 분석해 키워드 추출 중...
#  -> 🌟 Google Books/LLM 기반 추출 키워드: ['주식시장', '투자전략', '검증된 방법', '투자', '전략들']

# [System] 도서관 보유 장서(30권) 벡터화 시작...
#  -> 진행률: 49/49 (부자 아빠 가난한 아빠. 2, '현금흐름 사분면’과 돈을 관리하는 7가지 방법)
# [System] 벡터화 완료! ✅

# -----------------------------------------
# 테스트할 후기를 입력하세요 (예: 너무 어렵고 지루했다): 포트폴리오가 중요하다고 생각했고 사회초년생에게 추천합니다

# [LLM Thinking] 사용자 리뷰 분석 중... ('포트폴리오가 중요하다고 생각했고 사회초년생에게 추천합니다')
#  -> ✅ 사용자 취향 키워드 변경됨: ['주식시장', '투자전략', '포트폴리오', '투자', '사회초년생 투자']

# [Algorithm] 벡터 유사도 계산 및 필터링 시작...
#  -> 사용자 쿼리: 장르='경제학', 키워드=['주식시장', '투자전략', '포트폴리오', '투자', '사회초년생 투자']

# --- [Debug] 유사도 Top 3 후보 ---
#  1위: 사랑 사랑 사랑하는 이유 (0.9995)
#  2위: 딸에 대하여 : 김혜진 장편소설 (0.9995)
#  3위: 한국이 싫어서 : 장강명 장편소설 (0.9995)

# =========================================
#        🎉 최종 추천 결과 (3권)
# =========================================
# [Special Pick] 딸에 대하여 : 김혜진 장편소설
#  - 선정 논리: 장르 매칭 실패 -> 상위권 중 가장 오래된 책(2024-10-26)
#  - 벡터 유사도: 0.9995
# -----------------------------------------
# [Similarity Pick] 사랑 사랑 사랑하는 이유
#  - 선정 논리: 단순 벡터 유사도 최상위
#  - 벡터 유사도: 0.9995
# -----------------------------------------
# [Similarity Pick] 한국이 싫어서 : 장강명 장편소설
#  - 선정 논리: 단순 벡터 유사도 최상위
#  - 벡터 유사도: 0.9995
# -----------------------------------------

In [ ]:
from google.colab import data_table # 코랩 전용 매직 라이브러리

In [ ]:
# 1. 데이터프레임 생성
df_check = pd.DataFrame(LIBRARY_DB)

# [꿀팁] 키워드가 리스트(['a', 'b'])로 되어있으면 보기 싫으므로,
# 콤마(,)로 이어진 문자열("a, b")로 바꿔서 보여주는 게 훨씬 깔끔합니다.
df_display = df_check.copy()
df_display['keywords'] = df_display['keywords'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# 2. 보고 싶은 컬럼만 선택
target_cols = ['title', 'genre', 'keywords']

# 3. 마법의 코드: 인터랙티브 모드 켜기
data_table.enable_dataframe_formatter()

# print() 대신 그냥 변수명만 적거나 display()를 쓰세요!
print("⬇️ 아래 테이블은 클릭해서 정렬하거나 검색할 수 있습니다!")
display(df_display[target_cols])

# (참고) 다시 원래대로 텍스트로 보고 싶으면:
# data_table.disable_dataframe_formatter()

⬇️ 아래 테이블은 클릭해서 정렬하거나 검색할 수 있습니다!


,title,genre,keywords
0,사랑 사랑 사랑하는 이유,영미문학,"영미문학, 사랑, 사랑, 사랑하는, 이유"
1,사랑이 다예요,한국문학,"한국문학, 사랑이, 다예요"
2,억새풀. 5,한국문학,"한국문학, 억새풀, 5"
3,이번 생에 정신과는 처음이라 : 정신과 전문의가 말하는 정신과 사용 설명서,의학,"의학, 이번, 생에, 정신과는, 처음이라, 정신과"
4,강(江)이 쓰는 시(詩) : 낙동강 연작 제5 시조집,한국문학,"한국문학, 강江이, 쓰는, 시詩, 낙동강, 연작"
5,"(퀴즈!) 과학상식. 41, 별난요리",자연과학,"자연과학, 퀴즈, 과학상식, 41, 별난요리"
6,딸에 대하여 : 김혜진 장편소설,한국문학,"한국문학, 딸에, 대하여, 김혜진, 장편소설"
7,한국이 싫어서 : 장강명 장편소설,한국문학,"한국문학, 한국이, 싫어서, 장강명, 장편소설"
8,기도 : 당신과 당신이 사랑하는 모든 것을 지키는 힘,불교,"불교, 기도, 당신과, 당신이, 사랑하는, 모든"
9,(나를 돌보고 남을 살리는) 비건한 미식가 : 초식마녀 식탁 에세이,한국문학,"한국문학, 나를, 돌보고, 남을, 살리는, 비건한"
